In [2]:
!diamond makedb \
  --in proteins/Streptomyces_sp._MMCC_100.proteins.faa \
  --db g1.dmnd \
  --threads 8 --quiet

!diamond makedb \
  --in proteins/Streptomyces_rubrogriseus_NBRC_15455.proteins.faa \
  --db g2.dmnd \
  --threads 8 --quiet

In [3]:
#--max-target-seqs 1 \ (in rb for diamond its not used but used for blastp)
!diamond blastp \
  --threads 8 \
  --db g2.dmnd \
  --query proteins/Streptomyces_sp._MMCC_100.proteins.faa \
  --sensitive \
  --outfmt 6 \
  --out g1_vs_g2.tab \
  --quiet

#--max-target-seqs 1 \
!diamond blastp \
  --threads 8 \
  --db g1.dmnd \
  --query proteins/Streptomyces_rubrogriseus_NBRC_15455.proteins.faa \
  --sensitive \
  --outfmt 6 \
  --out g2_vs_g1.tab \
  --quiet

In [4]:
import pandas as pd
from Bio import SeqIO

def get_lengths(fasta):
    return {rec.id: len(rec.seq) for rec in SeqIO.parse(fasta, "fasta")}

len1 = get_lengths("proteins/Streptomyces_sp._MMCC_100.proteins.faa")
len2 = get_lengths("proteins/Streptomyces_rubrogriseus_NBRC_15455.proteins.faa")

print("Genome1 proteins:", len(len1))
print("Genome2 proteins:", len(len2))

Genome1 proteins: 7620
Genome2 proteins: 7577


In [5]:
cols = ["qseqid","sseqid","pident","length","mismatch","gapopen","qstart","qend","sstart","send","evalue","bitscore"]

df1 = pd.read_csv("g1_vs_g2.tab", sep="\t", header=None, names=cols)
df2 = pd.read_csv("g2_vs_g1.tab", sep="\t", header=None, names=cols)

In [6]:
#Keep only BEST hit per query
df1_best = df1.sort_values("bitscore", ascending=False).drop_duplicates("qseqid")
df2_best = df2.sort_values("bitscore", ascending=False).drop_duplicates("qseqid")


In [7]:
defaults = {"bits": 0,            # -s / --bitscore, Minimum bit score (default: 0)
    "id": 20,             # -i / --id, Minimum percent identity (default: 20%)
    "len": 0,             # -l / --len, Minim, Minimum number of reciprocal best hits required, to report two-way AAI (default: 50)
    "bin": "",            # -b / --bin, Path to directory containing search program binaries
    "program": "blast+",  # -p / --program, Search program used (blast+, blast, blat, diamond)
    "thr": 1,             # -t / --threads, Number of parallel threads (default: 1)
    "dec": 2,             # -d / --dec, Decimal positions to report (default: 2)
    "auto": False,        # -a / --auto, Only print AAI value (no detailed output)
    "lookupfirst": False, # --lookup-first, Lookup AAI in SQLite database before computing
    "dbrbm": True,        # --[no-]save-rbm, Save reciprocal best matches to SQLite database
    "nucl": False,        # -N / --nucl, Input sequences are nucleotides (genes), not proteins
    "len_fraction": 0.0,  # -L / --len-fraction, Minimum alignment length as fraction (0–1), of the shorter sequence
    "max_actg": 0.95,     # --max-actg, Max fraction of ACTGN before assuming nucleotide input
    "q": False            # -q / --quiet, Run quietly (no STDERR output)
}


MIN_ID = 20
MIN_LEN = 0
MIN_BITS = 0
LEN_FRAC = 0.0   # change this to your -L value


df1_best = df1_best[
    (df1_best["pident"] >= MIN_ID) &
    (df1_best["length"] >= MIN_LEN) &
    (df1_best["bitscore"] >= MIN_BITS)
]

df2_best = df2_best[
    (df2_best["pident"] >= MIN_ID) &
    (df2_best["length"] >= MIN_LEN) &
    (df2_best["bitscore"] >= MIN_BITS)
]

In [8]:
#Calculate len_fraction
#Get query protein lengths
df1_best["qlen"] = df1_best["qseqid"].map(len1)
df2_best["qlen"] = df2_best["qseqid"].map(len2)

#Get subject protein lengths
df1_best["slen"] = df1_best["sseqid"].map(len2)
df2_best["slen"] = df2_best["sseqid"].map(len1)

#Compute minimum protein length
df1_best["min_len"] = df1_best[["qlen", "slen"]].min(axis=1)
df2_best["min_len"] = df2_best[["qlen", "slen"]].min(axis=1)

#Compute fraction
df1_best["len_fraction"] = df1_best["length"] / df1_best["min_len"]
df2_best["len_fraction"] = df2_best["length"] / df2_best["min_len"]

#Apply -L filter
df1_best = df1_best[df1_best["len_fraction"] >= LEN_FRAC]
df2_best = df2_best[df2_best["len_fraction"] >= LEN_FRAC]

n1 = len(df1_best)
n2 = len(df2_best)

print("One-way hits Genome1→Genome2:", n1)
print("One-way hits Genome2→Genome1:", n2)


One-way hits Genome1→Genome2: 6756
One-way hits Genome2→Genome1: 6792


In [9]:
import numpy as np

# ---- One-way AAI Genome1 -> Genome2 ----
id1 = df1_best["pident"].sum()
sq1 = (df1_best["pident"] ** 2).sum()
n1 = len(df1_best)

if n1 > 0:
    aai1 = id1 / n1
    sd1 = np.sqrt((sq1 / n1) - (aai1 ** 2))
else:
    aai1 = 0
    sd1 = 0

print(f"One-way AAI 1: {aai1:.2f}% (SD: {sd1:.2f}%), from {n1} proteins.")


# ---- One-way AAI Genome2 -> Genome1 ----
id2 = df2_best["pident"].sum()
sq2 = (df2_best["pident"] ** 2).sum()
n2 = len(df2_best)

if n2 > 0:
    aai2 = id2 / n2
    sd2 = np.sqrt((sq2 / n2) - (aai2 ** 2))
else:
    aai2 = 0
    sd2 = 0

print(f"One-way AAI 2: {aai2:.2f}% (SD: {sd2:.2f}%), from {n2} proteins.")


One-way AAI 1: 84.62% (SD: 20.28%), from 6756 proteins.
One-way AAI 2: 83.76% (SD: 21.08%), from 6792 proteins.


In [10]:
# Create lookup for genome1 best hits
rbh_1 = dict(zip(df1_best["qseqid"], df1_best["sseqid"]))

# Create lookup for genome2 best hits
rbh_2 = dict(zip(df2_best["qseqid"], df2_best["sseqid"]))

# Find reciprocal matches
reciprocal_pairs = []

for q1, s1 in rbh_1.items():
    # if subject in genome2 points back to q1
    if s1 in rbh_2 and rbh_2[s1] == q1:
        reciprocal_pairs.append((q1, s1))

print("Reciprocal pairs:", len(reciprocal_pairs))


MIN_HITS = 50   # same as Ruby default -n 50

# ---- Reciprocal AAI ----
# Extract reciprocal rows from df1_best
df_rbm = df1_best[
    df1_best.apply(lambda x: (x["qseqid"], x["sseqid"]) in reciprocal_pairs, axis=1)
]

n_r = len(df_rbm)

if n_r < MIN_HITS:
    print(f"Insufficient hits to estimate two-way AAI: {n_r}")
else:
    id_r = df_rbm["pident"].sum()
    sq_r = (df_rbm["pident"] ** 2).sum()

    aai_r = id_r / n_r
    sd_r = np.sqrt((sq_r / n_r) - (aai_r ** 2))

    print(f"Two-way AAI: {aai_r:.2f}% (SD: {sd_r:.2f}%), from {n_r} proteins.")


Reciprocal pairs: 5757
Two-way AAI: 91.24% (SD: 11.21%), from 5757 proteins.
